<ul>
    <li> <h2><b>Universidade de Brasília - UnB</b></h2> </li>
    <li> <b>Campus UnB Gama</b> </li>
    <li> <b>Disciplina: Processamento Digital de Sinais Financeiros</b> </li>
    <a href="https://www.linkedin.com/in/marcelino-andrade-b164b369/">
    <li> <b>Prof. Marcelino Monteiro de Andrade Dr.</b> </li>
</ul>


    Primeira (1/5) Atividade Prática Avaliativa
    Entrega da Atividade 01/10/2019 
    "A não entrega na data esperada reduz em 50% na nota da avaliação"


<div id="Tópicos">
    <h2>Questões</h2>
</div>


<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol> 
        <li><a href="#Tópico 0">Questão 1</a></li>
        <li><a href="#Tópico 1">Questão 2</a></li>
        <li><a href="#Tópico 2">Questão 3</a></li>
        <li><a href="#Tópico 3">Questão 4</a></li>

</ol>
</div>


<div id="Tópico 0">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 1: Considerando os índices do grupo Setorial ou por Segmento, escolha 5 (cinco) índices e colete os preços diários (OHLC), assim como os volumes, dos 3 (três) papéis mais negociados (líquidos) em cada um dos 5 índices durante um período de 24 meses.
</h></a></h>   
</div> </div> 
http://www.bmfbovespa.com.br/pt_br/produtos/indices/

In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mpl_finance import candlestick_ohlc
import datetime
import pandas_datareader.data as pdr
import pickle
import datetime as dt
from scipy import signal


In [2]:
import matplotlib.dates as mdates
import datetime
from pandas import *
import matplotlib.mlab as mlab
import scipy.signal as signal
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import yfinance as yf
import datetime as dt

def yahoo(symbols,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year):
    yf.pdr_override()       
    stocks_yahoo = pdr.get_data_yahoo(symbols, start=datetime(S_Year, S_Month, S_Day), end=datetime(E_Year, E_Month, E_Day),   as_panel = False,)
    return stocks_yahoo

def retira_colunas_nan(df):
    newdf = None
    for i in range(len(df['Close'].columns)):
        if df['Close'].iloc[:,i].isna().sum() < 100:
            newdf=pd.concat([newdf,df['Close'].iloc[:,i]],axis=1) 
    return newdf

def ordena_acoes_por_diferenca_de_preco(df):
    resultado = {}
    for i in range(len(df.columns)):
        diferenca = df.iloc[-1,i] - df.iloc[0,i]
        resultado.update({df.iloc[0:i].columns[i]: diferenca})
        
    dict_ordered = sorted(resultado.items(), key=lambda x: x[1], reverse=True)
    
    for i in dict_ordered.value:
        if dict_ordered[i]<0:
            del dict_ordered[i]
            
    print(dict_ordered)

def cria_stocks_all(df):
    Stocks=df.values[:-2]
    N_months=24
    Stocks=[Stocks[k]+'.SA' for k in range(len(Stocks))]
    
    today = dt.date.today()
    E_Day = today.day; E_Month = today.month; E_Year = today.year
    day_before=today - dt.timedelta(days=N_months*365/12)
    S_Day = day_before.day; S_Month = day_before.month; S_Year = day_before.year

    stocks_all=yahoo(Stocks,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year)

    return stocks_all

def seleciona_acoes_crescentes(df):
    resultado = {}
    for i in range(len(df.columns)):
        diferenca = df.iloc[-1,i] - df.iloc[0,i]
        if diferenca > 0:
            resultado.update({df.iloc[0:i].columns[i]: diferenca})
        
    dict_crescentes = sorted(resultado.items(), key=lambda x: x[1], reverse=True) 

    return dict_crescentes

def seleciona_ativos_por_volume(dict_crescentes, stocks_all):
    dict_volume = {}
    for i in range(len(dict_crescentes)):
        dict_volume.update({dict_crescentes[i][0]: stocks_all['Volume'][dict_crescentes[i][0]].sum()}) 

    dict_volume_ordered = sorted(dict_volume.items(), key=lambda x: x[1], reverse=True) 
    dict_volume_ordered[:3]
    
    nome_acoes = []
    for i in dict_volume_ordered[:3]:
        nome_acoes.append(i[0])
        
    return nome_acoes


In [3]:
#Os dados foram baixados no site http://www.bmfbovespa.com.br/pt_br/produtos/indices/

# Lendo dados do indice IMOB
dfIMOB = pd.read_csv('C:/Users/bruno/Desktop/IMOBdata.csv')
dfIMOB = dfIMOB['Código\xa0']

stocks_all_IMOB = cria_stocks_all(dfIMOB)
stocks_all_IMOB_clean = retira_colunas_nan(stocks_all_IMOB)
dict_crescentes_IMOB = seleciona_acoes_crescentes(stocks_all_IMOB_clean)
nome_acoes_IMOB = seleciona_ativos_por_volume(dict_crescentes_IMOB, stocks_all_IMOB)

df_selecionadas_IMOB = pd.DataFrame()
for i in range(len(nome_acoes_IMOB)):
    df_selecionadas_IMOB = pd.concat([df_selecionadas_IMOB,stocks_all_IMOB['Close'][nome_acoes_IMOB[i]]],axis=1) 

df_selecionadas_IMOB

[*********************100%***********************]  14 of 14 downloaded


,BRML3.SA,MRVE3.SA,CYRE3.SA
2017-09-29,14.07,12.51,13.80
2017-10-02,14.09,12.45,13.76
2017-10-03,14.39,12.77,14.12
2017-10-04,14.30,12.96,14.07
2017-10-05,13.94,12.67,14.33
2017-10-06,13.80,12.71,14.01
2017-10-09,13.90,12.75,13.80
2017-10-10,13.97,12.91,14.21
2017-10-11,14.01,13.01,14.33
2017-10-12,14.01,13.01,14.33


In [4]:
# Lendo dados do indice IDIV
dfIDIV = pd.read_csv('C:/Users/bruno/Desktop/IDIVdata.csv')
dfIDIV = dfIDIV['Código\xa0']

stocks_all_IDIV = cria_stocks_all(dfIDIV)
stocks_all_IDIV_clean = retira_colunas_nan(stocks_all_IDIV)
dict_crescentes_IDIV = seleciona_acoes_crescentes(stocks_all_IDIV_clean)
nome_acoes_IDIV = seleciona_ativos_por_volume(dict_crescentes_IDIV,stocks_all_IDIV)

df_selecionadas_IDIV = pd.DataFrame()
for i in range(len(nome_acoes_IDIV)):
     df_selecionadas_IDIV = pd.concat([df_selecionadas_IDIV,stocks_all_IDIV['Close'][nome_acoes_IDIV[i]]],axis=1) 
    
df_selecionadas_IDIV


[*********************100%***********************]  29 of 29 downloaded


,ITSA4.SA,ITUB4.SA,BBSE3.SA
2017-09-29,10.03,28.90,28.63
2017-10-02,9.96,28.94,28.29
2017-10-03,10.35,29.93,28.52
2017-10-04,10.28,29.67,29.12
2017-10-05,10.35,30.14,29.30
2017-10-06,10.30,29.79,29.05
2017-10-09,10.15,29.43,29.00
2017-10-10,10.35,30.07,29.12
2017-10-11,10.33,30.00,29.10
2017-10-12,10.33,30.00,29.10


In [5]:
# Lendo dados do indice ICON
dfICON = pd.read_csv('C:/Users/bruno/Desktop/ICONdata.csv')
dfICON = dfICON['Código\xa0']

stocks_all_ICON = cria_stocks_all(dfICON)
stocks_all_ICON_clean = retira_colunas_nan(stocks_all_ICON)
dict_crescentes_ICON = seleciona_acoes_crescentes(stocks_all_ICON_clean)
nome_acoes_ICON = seleciona_ativos_por_volume(dict_crescentes_ICON, stocks_all_ICON)

df_selecionadas_ICON = pd.DataFrame()
for i in range(len(nome_acoes_ICON)):
    df_selecionadas_ICON = pd.concat([df_selecionadas_ICON,stocks_all_ICON['Close'][nome_acoes_ICON[i]]],axis=1) 
    
df_selecionadas_ICON


[*********************100%***********************]  54 of 54 downloaded


,MGLU3.SA,JBSS3.SA,LAME4.SA
2017-09-29,9.28,8.50,19.25
2017-10-02,9.73,8.67,19.12
2017-10-03,9.87,8.80,19.64
2017-10-04,9.94,8.78,20.20
2017-10-05,9.80,8.77,20.35
2017-10-06,9.96,8.57,20.01
2017-10-09,9.72,8.42,19.72
2017-10-10,9.95,8.49,19.94
2017-10-11,9.62,8.60,19.86
2017-10-12,9.62,8.60,19.86


In [6]:
# Lendo dados do indice IEE
dfIEE = pd.read_csv('C:/Users/bruno/Desktop/IEEdata.csv')
dfIEE = dfIEE['Código\xa0']

stocks_all_IEE = cria_stocks_all(dfIEE)
stocks_all_IEE_clean = retira_colunas_nan(stocks_all_IEE)
dict_crescentes_IEE = seleciona_acoes_crescentes(stocks_all_IEE_clean)
nome_acoes_IEE = seleciona_ativos_por_volume(dict_crescentes_IEE, stocks_all_IEE)

df_selecionadas_IEE = pd.DataFrame()
for i in range(len(nome_acoes_IEE)):
    df_selecionadas_IEE = pd.concat([df_selecionadas_IEE,stocks_all_IEE['Close'][nome_acoes_IEE[i]]],axis=1) 
    
df_selecionadas_IEE


[*********************100%***********************]  15 of 15 downloaded


,CMIG4.SA,ELET3.SA,ENBR3.SA
2017-09-29,7.74,19.76,15.14
2017-10-02,7.72,20.54,15.00
2017-10-03,8.16,21.80,15.16
2017-10-04,7.96,21.85,15.37
2017-10-05,8.13,21.47,15.40
2017-10-06,8.01,20.81,15.35
2017-10-09,7.99,20.72,15.24
2017-10-10,8.09,21.30,15.59
2017-10-11,8.10,21.03,15.49
2017-10-12,8.10,21.03,15.49


In [7]:
# Lendo dados do indice INDX
dfINDX = pd.read_csv('C:/Users/bruno/Desktop/INDXdata.csv')
dfINDX = dfINDX['Código\xa0']

stocks_all_INDX = cria_stocks_all(dfINDX)
stocks_all_INDX_clean = retira_colunas_nan(stocks_all_INDX)
dict_crescentes_INDX = seleciona_acoes_crescentes(stocks_all_INDX_clean)
nome_acoes_INDX = seleciona_ativos_por_volume(dict_crescentes_INDX, stocks_all_INDX)

df_selecionadas_INDX = pd.DataFrame()
for i in range(len(nome_acoes_INDX)):
    df_selecionadas_INDX = pd.concat([df_selecionadas_INDX,stocks_all_INDX['Close'][nome_acoes_INDX[i]]],axis=1) 
    
df_selecionadas_INDX


[*********************100%***********************]  41 of 41 downloaded


,USIM5.SA,GOAU4.SA,GGBR4.SA
2017-09-29,7.78,5.29,11.04
2017-10-02,7.89,5.33,11.00
2017-10-03,8.66,5.62,11.40
2017-10-04,9.02,5.70,11.37
2017-10-05,8.96,5.45,11.08
2017-10-06,9.35,5.34,10.76
2017-10-09,9.18,5.37,10.70
2017-10-10,9.45,5.66,11.15
2017-10-11,9.70,5.57,11.02
2017-10-12,9.70,5.57,11.02


In [8]:
#MATRIZ DE CORRELAÇÃO COM OS 15 ATIVOS
df_selecionadas = pd.concat([df_selecionadas_IEE,df_selecionadas_INDX,df_selecionadas_ICON,df_selecionadas_IDIV,df_selecionadas_IMOB],axis=1)
Correlacao15 = df_selecionadas.corr()
Correlacao15

,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GOAU4.SA,GGBR4.SA,MGLU3.SA,JBSS3.SA,LAME4.SA,ITSA4.SA,ITUB4.SA,BBSE3.SA,BRML3.SA,MRVE3.SA,CYRE3.SA
CMIG4.SA,1.000000,0.911584,0.848725,-0.170334,0.116065,0.034846,0.860675,0.803122,0.319391,0.776268,0.783120,0.524720,0.662045,0.634863,0.794048
ELET3.SA,0.911584,1.000000,0.926163,-0.132369,-0.078337,-0.165951,0.830740,0.882217,0.286405,0.780151,0.742111,0.721663,0.778765,0.796028,0.919906
ENBR3.SA,0.848725,0.926163,1.000000,-0.311069,-0.186099,-0.258918,0.811758,0.906872,0.135897,0.620771,0.567961,0.676256,0.708787,0.838784,0.873406
USIM5.SA,-0.170334,-0.132369,-0.311069,1.000000,0.358258,0.374801,-0.405287,-0.376867,0.278312,0.330411,0.357185,0.121949,0.089121,-0.099240,-0.095093
GOAU4.SA,0.116065,-0.078337,-0.186099,0.358258,1.000000,0.974646,0.128369,-0.118216,0.241491,0.329096,0.364057,-0.258752,-0.363228,-0.127072,-0.114577
GGBR4.SA,0.034846,-0.165951,-0.258918,0.374801,0.974646,1.000000,0.041169,-0.191297,0.211453,0.269518,0.297018,-0.317261,-0.438844,-0.183652,-0.196863
MGLU3.SA,0.860675,0.830740,0.811758,-0.405287,0.128369,0.041169,1.000000,0.901528,0.263817,0.630177,0.594846,0.530308,0.493390,0.702045,0.829892
JBSS3.SA,0.803122,0.882217,0.906872,-0.376867,-0.118216,-0.191297,0.901528,1.000000,0.065722,0.609202,0.532090,0.725056,0.603707,0.862264,0.911206
LAME4.SA,0.319391,0.286405,0.135897,0.278312,0.241491,0.211453,0.263817,0.065722,1.000000,0.544003,0.550903,0.215311,0.376256,0.078421,0.274779
ITSA4.SA,0.776268,0.780151,0.620771,0.330411,0.329096,0.269518,0.630177,0.609202,0.544003,1.000000,0.978343,0.673306,0.657066,0.644095,0.765011


In [9]:
#NOME DOS 15 ATIVOS SELECIONADOS
nome_acoes_selecionadas = nome_acoes_IEE + nome_acoes_INDX + nome_acoes_ICON + nome_acoes_IDIV + nome_acoes_IMOB
nome_acoes_selecionadas

['CMIG4.SA',
 'ELET3.SA',
 'ENBR3.SA',
 'USIM5.SA',
 'GOAU4.SA',
 'GGBR4.SA',
 'MGLU3.SA',
 'JBSS3.SA',
 'LAME4.SA',
 'ITSA4.SA',
 'ITUB4.SA',
 'BBSE3.SA',
 'BRML3.SA',
 'MRVE3.SA',
 'CYRE3.SA']

In [15]:
#CRIAR ITERAÇÕES DAS POSSÍVEIS CARTEIRAS
from itertools import combinations 
comb = combinations(nome_acoes_selecionadas, 5) 
COMB_15_5_ACOES = pd.DataFrame(list(comb))
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Ação1",1:"Ação2",2: "Ação3",3: "Ação4",4: "Ação5"}) 
COMB_15_5_ACOES

,Ação1,Ação2,Ação3,Ação4,Ação5
0,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GOAU4.SA
1,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GGBR4.SA
2,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,MGLU3.SA
3,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,JBSS3.SA
4,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,LAME4.SA
5,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,ITSA4.SA
6,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,ITUB4.SA
7,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,BBSE3.SA
8,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,BRML3.SA
9,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,MRVE3.SA


In [16]:
#BUSCAR CORRELAÇÃO 2 A 2 
Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação1'][i]][COMB_15_5_ACOES['Ação2'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr1"})


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação1'][i]][COMB_15_5_ACOES['Ação3'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr2"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação1'][i]][COMB_15_5_ACOES['Ação4'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr3"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação1'][i]][COMB_15_5_ACOES['Ação5'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr4"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação2'][i]][COMB_15_5_ACOES['Ação3'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr5"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação2'][i]][COMB_15_5_ACOES['Ação4'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr6"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação2'][i]][COMB_15_5_ACOES['Ação5'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr7"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação3'][i]][COMB_15_5_ACOES['Ação4'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr8"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação3'][i]][COMB_15_5_ACOES['Ação5'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr9"})
COMB_15_5_ACOES


Coluna_corr = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Coluna_corr.append(Correlacao15[COMB_15_5_ACOES['Ação4'][i]][COMB_15_5_ACOES['Ação5'][i]])

Coluna_corr = pd.DataFrame(Coluna_corr)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Coluna_corr],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Corr10"})
COMB_15_5_ACOES

,Ação1,Ação2,Ação3,Ação4,Ação5,Corr1,Corr2,Corr3,Corr4,Corr5,Corr6,Corr7,Corr8,Corr9,Corr10
0,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GOAU4.SA,0.911584,0.848725,-0.170334,0.116065,0.926163,-0.132369,-0.078337,-0.311069,-0.186099,0.358258
1,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GGBR4.SA,0.911584,0.848725,-0.170334,0.034846,0.926163,-0.132369,-0.165951,-0.311069,-0.258918,0.374801
2,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,MGLU3.SA,0.911584,0.848725,-0.170334,0.860675,0.926163,-0.132369,0.830740,-0.311069,0.811758,-0.405287
3,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,JBSS3.SA,0.911584,0.848725,-0.170334,0.803122,0.926163,-0.132369,0.882217,-0.311069,0.906872,-0.376867
4,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,LAME4.SA,0.911584,0.848725,-0.170334,0.319391,0.926163,-0.132369,0.286405,-0.311069,0.135897,0.278312
5,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,ITSA4.SA,0.911584,0.848725,-0.170334,0.776268,0.926163,-0.132369,0.780151,-0.311069,0.620771,0.330411
6,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,ITUB4.SA,0.911584,0.848725,-0.170334,0.783120,0.926163,-0.132369,0.742111,-0.311069,0.567961,0.357185
7,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,BBSE3.SA,0.911584,0.848725,-0.170334,0.524720,0.926163,-0.132369,0.721663,-0.311069,0.676256,0.121949
8,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,BRML3.SA,0.911584,0.848725,-0.170334,0.662045,0.926163,-0.132369,0.778765,-0.311069,0.708787,0.089121
9,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,MRVE3.SA,0.911584,0.848725,-0.170334,0.634863,0.926163,-0.132369,0.796028,-0.311069,0.838784,-0.099240


In [17]:
#CÁLCULO DA MÉDIA E DESVIO PADRÃO DAS CORRELAÇÕES DAS CARTEIRAS
Media = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Media.append(abs(COMB_15_5_ACOES.loc[i][5:15]).mean())

Media = pd.DataFrame(Media)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Media],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Média"})

COMB_15_5_ACOES

Desv = []
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    Desv.append(COMB_15_5_ACOES.loc[i][5:15].std())

Desv = pd.DataFrame(Desv)
COMB_15_5_ACOES = pd.concat([COMB_15_5_ACOES,Desv],axis=1) 
COMB_15_5_ACOES = COMB_15_5_ACOES.rename(columns = {0: "Desvio"})
COMB_15_5_ACOES

,Ação1,Ação2,Ação3,Ação4,Ação5,Corr1,Corr2,Corr3,Corr4,Corr5,Corr6,Corr7,Corr8,Corr9,Corr10,Média,Desvio
0,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GOAU4.SA,0.911584,0.848725,-0.170334,0.116065,0.926163,-0.132369,-0.078337,-0.311069,-0.186099,0.358258,0.403900,0.495980
1,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,GGBR4.SA,0.911584,0.848725,-0.170334,0.034846,0.926163,-0.132369,-0.165951,-0.311069,-0.258918,0.374801,0.413476,0.512656
2,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,MGLU3.SA,0.911584,0.848725,-0.170334,0.860675,0.926163,-0.132369,0.830740,-0.311069,0.811758,-0.405287,0.620870,0.583773
3,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,JBSS3.SA,0.911584,0.848725,-0.170334,0.803122,0.926163,-0.132369,0.882217,-0.311069,0.906872,-0.376867,0.626932,0.587028
4,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,LAME4.SA,0.911584,0.848725,-0.170334,0.319391,0.926163,-0.132369,0.286405,-0.311069,0.135897,0.278312,0.432025,0.456542
5,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,ITSA4.SA,0.911584,0.848725,-0.170334,0.776268,0.926163,-0.132369,0.780151,-0.311069,0.620771,0.330411,0.580785,0.489925
6,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,ITUB4.SA,0.911584,0.848725,-0.170334,0.783120,0.926163,-0.132369,0.742111,-0.311069,0.567961,0.357185,0.575062,0.485426
7,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,BBSE3.SA,0.911584,0.848725,-0.170334,0.524720,0.926163,-0.132369,0.721663,-0.311069,0.676256,0.121949,0.534483,0.485971
8,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,BRML3.SA,0.911584,0.848725,-0.170334,0.662045,0.926163,-0.132369,0.778765,-0.311069,0.708787,0.089121,0.553896,0.499852
9,CMIG4.SA,ELET3.SA,ENBR3.SA,USIM5.SA,MRVE3.SA,0.911584,0.848725,-0.170334,0.634863,0.926163,-0.132369,0.796028,-0.311069,0.838784,-0.099240,0.566916,0.527267


In [11]:
#Excluir as carteiras com média de correlação superior a 0,3
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    if(COMB_15_5_ACOES['Média'][i] > 0.3):
        COMB_15_5_ACOES.drop(i, inplace=True, axis=0)

COMB_15_5_ACOES.reset_index(inplace = True, drop = True)

#Excluir as carteiras com desvio padrão de correlação superior a 0,3
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    if(COMB_15_5_ACOES['Desvio'][i] > 0.3):
        COMB_15_5_ACOES.drop(i, inplace=True, axis=0)

COMB_15_5_ACOES.reset_index(inplace = True, drop = True)

#SELECIONA A CARTEIRA COM MENOR MÉDIA
MEDIA = 1
for i in range(len(COMB_15_5_ACOES['Ação1'])):
    if(COMB_15_5_ACOES['Média'][i] < MEDIA):
        MEDIA = COMB_15_5_ACOES['Média'][i]
        MEDIA_index = i

CARTEIRA_FINAL_SELECIONADA = COMB_15_5_ACOES.loc[MEDIA_index][0:5]
CARTEIRA_FINAL_SELECIONADA

Ação1    CMIG4.SA
Ação2    USIM5.SA
Ação3    GGBR4.SA
Ação4    LAME4.SA
Ação5    MRVE3.SA
Name: 5, dtype: object

<div id="Tópico 1">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 2: Identifique a correlação entre os preços e os retornos dos 15 (quinze) ativos coletados na questão anterior. Em seguida, identifique os cinco papeis com menor correlação e retorno médio positivo no período de 24 meses.</h></a></h>   
</div> </div> 

In [12]:
a=1

<div id="Tópico 2">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 3: Com relação aos 5 (cinco) papéis da questão 2, apresente: (i) os gráficos de preço no formato candlestick, (ii) os gráficos de retorno mês, (iii) o índice Sharpe e o (iv) fator Beta com relação ao índice Bovespa.</h></a></h>   
</div> </div> 

In [13]:
a=1

<div id="Tópico 3">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 4: Considerando os 5 (cinco) papeis escolhidos, na questão 2, aplique a Teoria do Portfólio Moderno de Markowitz. Nesse sentido, (i) Identifique o diagrama Risco x Retorno, (ii) a curva de eficiência, (iii) o portfólio de menor risco e o (iv) portfólio com maior índice Sharpe.</h></a></h>   
</div> </div> 

In [14]:
a=1